In [90]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [91]:
x = np.load('data/EEG_all_epochs.npy')
y = np.load('data/EEG_contaminated_with_EOG/EEG_contaminated_with_EOG_2db.npy')

In [92]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.1, random_state = 42)

In [93]:
model = Sequential()
model.add(Dense(512, input_dim=y_train.shape[1], activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='linear'))

model.compile(loss = 'mean_squared_error', optimizer = 'Nadam')

model.fit(y_train, x_train, epochs = 300, batch_size = 32, verbose = 1)

Epoch 1/300
127/127 [==============================] - 2s 8ms/step - loss: 35069.1914
Epoch 2/300
127/127 [==============================] - 1s 7ms/step - loss: 16365.8994
Epoch 3/300
127/127 [==============================] - 1s 7ms/step - loss: 11928.5508
Epoch 4/300
127/127 [==============================] - 1s 7ms/step - loss: 9735.9590
Epoch 5/300
127/127 [==============================] - 1s 8ms/step - loss: 8498.2041
Epoch 6/300
127/127 [==============================] - 1s 7ms/step - loss: 7689.8677
Epoch 7/300
127/127 [==============================] - 1s 8ms/step - loss: 6953.4473
Epoch 8/300
127/127 [==============================] - 1s 8ms/step - loss: 6531.3853
Epoch 9/300
127/127 [==============================] - 1s 7ms/step - loss: 6142.9873
Epoch 10/300
127/127 [==============================] - 1s 7ms/step - loss: 5724.9648
Epoch 11/300
127/127 [==============================] - 1s 7ms/step - loss: 5484.7856
Epoch 12/300
127/127 [==============================] - 1s 7

In [94]:
y_pred = model.predict(x_val)

# Calcular el error cuadrático medio entre las predicciones y los datos de validación limpios
mse = mean_squared_error(y_val, y_pred)
print("MSE:", mse)

# Calcular el coeficiente de correlación entre las predicciones y los datos de validación limpios
corr = np.corrcoef(y_val.reshape(-1), y_pred.reshape(-1))[0, 1]
print("Coeficiente de correlación:", corr)

15/15 [==============================] - 0s 2ms/step
MSE: 31343.816246096976
Coeficiente de correlación: 0.7801909022120351


Adam: 0.60
Adamax: 0.61
Nadam: 0.61